<a href="https://colab.research.google.com/github/findQin/ColabCode/blob/master/mynlpLearn4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
# 修改colab笔记本设置，使用GPU
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
dataset = tfds.load('imdb_reviews/plain_text', as_supervised=True)
train_data, test_data = dataset['train'], dataset['test']

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteFF7YOL/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteFF7YOL/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteFF7YOL/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [4]:
# 取出两条数据看看
train_examples_batch, train_labels_batch = next(iter(train_data.batch(2)))

In [5]:
# tf.Tensor对象，评论存储在numpy属性中，b""也是对象
train_examples_batch

<tf.Tensor: shape=(2,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell a

In [6]:
# 标签存储在numpy属性中，可以看到前两天评论标签都是0 
train_labels_batch

<tf.Tensor: shape=(2,), dtype=int64, numpy=array([0, 0])>

In [7]:
# 将训练集数据取出，分别将数据与标签分开存放
x_train, y_train = next(iter(train_data.batch(25000)))
print("Train-set size: ", len(x_train))

Train-set size:  25000


In [8]:
# 还都是 tf.Tensor 对象
x_train[0]

<tf.Tensor: shape=(), dtype=string, numpy=b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.">

In [9]:
# 取出我们想要的文本，虽然还是b""，应该也是字节对象
x_train[0].numpy()

b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."

In [10]:
type(x_train[0].numpy())

bytes

In [11]:
x_train_bytes = x_train.numpy().tolist()

In [17]:
x_train_bytes[:3]

[b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
 b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development 

In [13]:
# 目的将字节型评论文本转为字符串型
x_train_data = []

for x in x_train_bytes:
  x_train_data.append(x.decode('utf-8'))

In [15]:
# 转为了字符串型
x_train_data[:3]

["This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
 'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development wa

In [16]:
len(x_train_data)

25000

In [18]:
# 一样处理测试集
x_test, y_test = next(iter(test_data.batch(25000)))
print("Test-set size:  ", len(x_test))

Test-set size:   25000


In [19]:
x_test_bytes = x_test.numpy().tolist()

In [20]:
x_test_data = []

for x in x_test_bytes:
  x_test_data.append(x.decode('utf-8'))

In [21]:
x_test_data[:3]

["There are films that make careers. For George Romero, it was NIGHT OF THE LIVING DEAD; for Kevin Smith, CLERKS; for Robert Rodriguez, EL MARIACHI. Add to that list Onur Tukel's absolutely amazing DING-A-LING-LESS. Flawless film-making, and as assured and as professional as any of the aforementioned movies. I haven't laughed this hard since I saw THE FULL MONTY. (And, even then, I don't think I laughed quite this hard... So to speak.) Tukel's talent is considerable: DING-A-LING-LESS is so chock full of double entendres that one would have to sit down with a copy of this script and do a line-by-line examination of it to fully appreciate the, uh, breadth and width of it. Every shot is beautifully composed (a clear sign of a sure-handed director), and the performances all around are solid (there's none of the over-the-top scenery chewing one might've expected from a film like this). DING-A-LING-LESS is a film whose time has come.",
 "A blackly comic tale of a down-trodden priest, Nazarin

In [22]:
# 开始设置分词器，num_words: 词汇表所要保存的词数，基于词频，保存前10000个频率最高的
# oov_token: 对于以后要转换的句子中不在词汇表中的词，以 oov_token 中的值表示
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(x_train_data)

In [24]:
# 词汇表
# tokenizer.word_index

# {'<OOV>': 1,
# 'the': 2,
# 'and': 3,
# 'a': 4,...}

In [25]:
# 根据词汇表中每个词对应的整数索引，将训练集每条文本中的词转换为整数索引
x_train_tokens = tokenizer.texts_to_sequences(x_train_data)

In [26]:
x_train_data[1]

'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development was constant. Constantly slow and boring. Things seemed to happen, but with no explanation of what was causing them or why. I admit, I may have missed part of the film, but i watched the majority of it and everything just seemed to happen of its own accord without any real concern for anything else. I cant recommend this film at all.'

In [32]:
x_train_tokens[1]

[11,
 26,
 75,
 571,
 6,
 805,
 2354,
 313,
 106,
 19,
 12,
 7,
 629,
 686,
 6,
 4,
 2219,
 5,
 181,
 584,
 64,
 1454,
 110,
 2263,
 3,
 3951,
 21,
 2,
 1,
 3,
 258,
 41,
 4677,
 4,
 174,
 188,
 21,
 12,
 4078,
 11,
 1578,
 2354,
 86,
 2,
 20,
 14,
 1907,
 2,
 112,
 940,
 14,
 1811,
 1340,
 548,
 3,
 355,
 181,
 466,
 6,
 591,
 19,
 17,
 55,
 1817,
 5,
 49,
 14,
 4044,
 96,
 40,
 136,
 11,
 972,
 11,
 201,
 26,
 1046,
 171,
 5,
 2,
 20,
 19,
 11,
 294,
 2,
 2155,
 5,
 10,
 3,
 283,
 41,
 466,
 6,
 591,
 5,
 92,
 203,
 1,
 207,
 99,
 145,
 4382,
 16,
 230,
 332,
 11,
 2486,
 384,
 12,
 20,
 31,
 30]

In [29]:
np.array(x_train_tokens[1])

array([  11,   26,   75,  571,    6,  805, 2354,  313,  106,   19,   12,
          7,  629,  686,    6,    4, 2219,    5,  181,  584,   64, 1454,
        110, 2263,    3, 3951,   21,    2,    1,    3,  258,   41, 4677,
          4,  174,  188,   21,   12, 4078,   11, 1578, 2354,   86,    2,
         20,   14, 1907,    2,  112,  940,   14, 1811, 1340,  548,    3,
        355,  181,  466,    6,  591,   19,   17,   55, 1817,    5,   49,
         14, 4044,   96,   40,  136,   11,  972,   11,  201,   26, 1046,
        171,    5,    2,   20,   19,   11,  294,    2, 2155,    5,   10,
          3,  283,   41,  466,    6,  591,    5,   92,  203,    1,  207,
         99,  145, 4382,   16,  230,  332,   11, 2486,  384,   12,   20,
         31,   30])

In [30]:
# 将测试集每条文本也转换为整数编码
x_test_tokens = tokenizer.texts_to_sequences(x_test_data)

In [33]:
# 训练集也好，测试集也好，其中每条文本的长度都不一致，
# 这里我们想将这一条条文本的长度通过填充、截取设为一致
# 根据所有文本的长度，取一个平均长度，超过其的截断，不够的则填充
# [1,2,3]+[4,5,6] -> [1,2,3,4,5,6]
# x_train_tokens + x_test_tokens 将两个列表合并为一个[[1,2,3],[4,5,6,7].[8,9],...]
num_tokens = [len(x_tokens) for x_tokens in x_train_tokens + x_test_tokens]

In [34]:
# 50000条句子长度已统计完
print(len(num_tokens))

50000


In [35]:
# 前三条句子长度分别为117，112，132
num_tokens[:3]

[117, 112, 132]

In [36]:
num_tokens = np.array(num_tokens)

In [37]:
# 所有句子长度的均值
np.mean(num_tokens)

235.03316

In [38]:
# 所有句子中最长的句子的长度
np.max(num_tokens)

2493

In [39]:
# 规定句子长度为：所有句子长度的均值加上所有句子长度的标准差的2倍
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens

583

In [40]:
# 所有句子中长度小于583的句子占到总数的95%
np.sum(num_tokens < max_tokens) / len(num_tokens)

0.94514

In [41]:
padding_type = 'pre'

In [42]:
# 句子最大长度max_token,超过的前向截断，不够的前向填充，
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens, padding=padding_type, truncating=padding_type)
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens, padding=padding_type, truncating=padding_type)

In [43]:
# 训练集和测试集各25000条数据。每条数据长度都修正为583
print("x_train_pad shape: ", x_train_pad.shape)
print("x_test_pad shape: ", x_test_pad.shape)

x_train_pad shape:  (25000, 583)
x_test_pad shape:  (25000, 583)


In [44]:
np.array(x_train_tokens[1])

array([  11,   26,   75,  571,    6,  805, 2354,  313,  106,   19,   12,
          7,  629,  686,    6,    4, 2219,    5,  181,  584,   64, 1454,
        110, 2263,    3, 3951,   21,    2,    1,    3,  258,   41, 4677,
          4,  174,  188,   21,   12, 4078,   11, 1578, 2354,   86,    2,
         20,   14, 1907,    2,  112,  940,   14, 1811, 1340,  548,    3,
        355,  181,  466,    6,  591,   19,   17,   55, 1817,    5,   49,
         14, 4044,   96,   40,  136,   11,  972,   11,  201,   26, 1046,
        171,    5,    2,   20,   19,   11,  294,    2, 2155,    5,   10,
          3,  283,   41,  466,    6,  591,    5,   92,  203,    1,  207,
         99,  145, 4382,   16,  230,  332,   11, 2486,  384,   12,   20,
         31,   30])

In [47]:
len(x_train_tokens[1])

112

In [45]:
x_train_pad[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [46]:
len(x_train_pad[1])

583

In [48]:
# word_index: 词汇表
# {'<OOV>': 1,
# 'the': 2,
# 'and': 3,
# 'a': 4,...}
word_index = tokenizer.word_index

# 词汇表键值对调
inverse_map = dict(zip(word_index.values(), word_index.keys()))

In [51]:
inverse_map[1]

'<OOV>'

In [52]:
inverse_map[2]

'the'

In [53]:
# 映射回去看看，将x_train_token或x_train_pad这些整数编码，根据反转词汇表转回文本看看
def tokens_to_string(tokens):
    # Map from tokens back to words, 0 is padding_type, no meaning 
    words = [inverse_map[token] for token in tokens if token != 0]
    
    # Concatenate all words.
    text = " ".join(words)

    return text

In [54]:
tokens_to_string(x_train_pad[1])

'i have been known to fall asleep during films but this is usually due to a combination of things including really tired being warm and comfortable on the <OOV> and having just eaten a lot however on this occasion i fell asleep because the film was rubbish the plot development was constant constantly slow and boring things seemed to happen but with no explanation of what was causing them or why i admit i may have missed part of the film but i watched the majority of it and everything just seemed to happen of its own <OOV> without any real concern for anything else i cant recommend this film at all'

In [55]:
x_train_data[1]

'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development was constant. Constantly slow and boring. Things seemed to happen, but with no explanation of what was causing them or why. I admit, I may have missed part of the film, but i watched the majority of it and everything just seemed to happen of its own accord without any real concern for anything else. I cant recommend this film at all.'

In [59]:
# 开始搭建模型
# 词向量维度
embedding_size = 8

# 词汇表大小
num_words=10000

In [57]:
model = Sequential()

In [60]:
model.add(Embedding(input_dim=num_words,
          output_dim=embedding_size,
          input_length=max_tokens,
          name='layer_embedding'))

In [61]:
model.add(GRU(units=16, return_sequences=True))
model.add(GRU(units=8, return_sequences=True))
model.add(GRU(units=4))
model.add(Dense(1, activation='sigmoid'))

In [62]:
# lr:学习率
optimizer = Adam(lr=1e-3)

In [63]:
model.compile(loss='binary_crossentropy',
       optimizer=optimizer,
       metrics=['accuracy'])

In [64]:
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_embedding (Embedding)  (None, 583, 8)            80000     
_________________________________________________________________
gru (GRU)                    (None, 583, 16)           1248      
_________________________________________________________________
gru_1 (GRU)                  (None, 583, 8)            624       
_________________________________________________________________
gru_2 (GRU)                  (None, 4)                 168       
_________________________________________________________________
dense (Dense)                (None, 1)                 5         
Total params: 82,045
Trainable params: 82,045
Non-trainable params: 0
_________________________________________________________________


In [65]:
model.fit(x_train_pad, y_train,
          validation_split=0.05, epochs=3, batch_size=64)

Epoch 1/3
372/372 [==============================] - 21s 57ms/step - loss: 0.4864 - accuracy: 0.7451 - val_loss: 0.3426 - val_accuracy: 0.8592
Epoch 2/3
372/372 [==============================] - 21s 56ms/step - loss: 0.2671 - accuracy: 0.8989 - val_loss: 0.2885 - val_accuracy: 0.8768
Epoch 3/3
372/372 [==============================] - 21s 56ms/step - loss: 0.2116 - accuracy: 0.9241 - val_loss: 0.2939 - val_accuracy: 0.8784


In [66]:
result = model.evaluate(x_test_pad, y_test)

782/782 [==============================] - 18s 22ms/step - loss: 0.3211 - accuracy: 0.8678


In [67]:
print("Accuracy: {0:.2%}".format(result[1]))

Accuracy: 86.78%
